In [1]:
import sys 
import os
import cobra
import libsbml
import pandas as pd
import copy
from pathlib import Path
import memote
import csv
import pytest
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
import path 
import datetime
import scipy.sparse as sp
from statsmodels.tsa.stattools import acf


from cobra import sampling
from cobra import Reaction

#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_manipulation  as mm


In [10]:
#This codeblock is to define some of the functions used for modelling


#Set solver to gurobi
config = cobra.Configuration()
config.solver = 'gurobi'

#Read 2-cell model
wt_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")
trans_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6


#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light


def generate_constraint(model,reaction, name, lb, ub):
    reaction_fex = model.reactions.get_by_id(name).flux_expression
    constraint = model.problem.Constraint(reaction_fex, lb=lb, ub=ub)
    constraint.name = name + '_constraint'
    model.add_cons_vars

def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp


#This function is used to set the inputs to the model used. 
def define_model_medium(model, co2, o2, ppfd, 
                        medium_dir='./misc/photo_medium.csv', no3=inf, h2o=inf, h=inf, 
                        nh4=inf, pi=inf):
    model_photo_media = mm.read_medium_csv(medium_dir, model)
    model_photo_media['EX_no3(e)'] = no3
    model_photo_media['EX_h2o(e)'] = h2o
    model_photo_media['EX_h(e)'] = h
    model_photo_media['EX_nh4(e)'] = nh4
    model_photo_media['EX_co2(e)'] = co2
    model_photo_media['EX_o2(e)'] = o2
    model_photo_media['EX_photonVis(e)'] = ppfd
    model_photo_media['EX_pi(e)'] = pi
    #Set set model medium as model
#     print('Added model medium')
    return model_photo_media

    
def turn_off_cofac_cycles(model, inact_dir='./misc/leaf_inactivated.tsv'):
    file = csv.reader(open(inact_dir), delimiter='\t')
    leaf_inactive_rxns = list()
    for rows in file:
        row_m = str()
        row_bs = str()
        for rxns in rows:
            row_m += str(rxns) + "_M"
            row_bs += str(rxns) + "_BS"
        leaf_inactive_rxns.append(row_m)
        leaf_inactive_rxns.append(row_bs)
        
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            rxns.bounds = (0,0)
#     print('Successfully turned off cofactor-cycling reactions')

    
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")
def add_tissue_constraints(model):
    #For input fluxes for light, we will set the flux ratio to 10:1 to reflect the anatomical proportions of our model ()
    
    BS_photon_import = model.reactions.PRISM_white_LED_BS
    M_photon_import = model.reactions.PRISM_white_LED_M

    #Set photon flux ratio to 10:1
    photon_flux = mm.set_fix_flux_ratio({M_photon_import.id:10, BS_photon_import.id:1},model)
    model.add_cons_vars(photon_flux)

    
    #UPDATE: Change CO2 intake to the M Cell instead rather than set a ratio, which is a better assumption overall. Assume na lang that external gasses are assimilated
    #Via the M cell.
    #From Morrison et al 2005 -- Lateral diffusion of Gases is unlikely to support photosynthesis due to the
    #assimilation of diffused CO2 in tissues prior to BS//
    model.reactions.CO2tex_BS.bounds = (0,0)
    model.reactions.O2tex_BS.bounds = (0,0)
    
    #UPDATE: This assumption does not hold considering that recent transcriptomic analysis confirms that 
    #the bundle sheath is involved in the assimilation of inorganic nutrients, including nitrogen (nitrates/ammonia), and 
    #Sulfates. In turn, this will be implemented by simply setting the exchanges to the M cell to 0. (Hua et al, 2021)
    model.reactions.SO3tex_M.bounds = (0,0)
    model.reactions.SO4tex_M.bounds = (0,0)
    model.reactions.NH4tex_M.bounds = (0,0)
    model.reactions.NO3tex_M.bounds = (0,0)
    
    #Model will also constraint H2O input to BS cell only as it is also assumed that BS tissue in rice is specialized for H2O transport (Hua et al. 2021)
    #There is a demand reaction naman for H2O for the M cell which is not connected to the BS H2Otex
    #Restrict H2O transport to be unidirectional from the BS cell
    model.reactions.H2Otex_M.bounds = (0, 0)
    model.reactions.h2o_pd.bounds = (-inf, 0)
    
    #need to turn off HCO import as the model incorrectly transfers fixed HCO to the BS cell via the common pool compartment
    model.reactions.HCO3tex_M.bounds = (0,0)
    model.reactions.HCO3tex_BS.bounds = (0,0)
    
    #No constraints will be implemented for H+ availability allowing the model to use protons on-demand.
    
    #Turn off other Demand reactions that may serve as sinks for the model except DM_Phloem_BS (Which represents the output of photoassimilate thru the BS cell
    model.reactions.DM_Phloem_M.bounds = (0,0)
    model.reactions.Straw_Biomass_M.bounds = (0,0)
    model.reactions.Straw_Biomass_BS.bounds = (0,0)
    model.reactions.Coleoptile_Biomass_M.bounds = (0,0)
    model.reactions.Coleoptile_Biomass_BS.bounds = (0,0)
    model.reactions.DM_Phloem_BS.bounds = (0, inf)
    

def add_enzyme_constraints(model, 
                           wt_pepc = 0, 
                           wt_mdh = 11.18, 
                           wt_nadp_me = 0.14, 
                           wt_ppdk=0.31,
                          wt_CA=7.5):
    
    
    # #This code block contains constraints specific for enzyme rate constraints
    #This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
    #fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
    #At 0. For reversible reactions the lower bounds are set to the same value
    
    
    #PEPC constraint (Reaction id: PPCc)
    #Need to constrain it to 0 since reaction is only detected in Vascular tissue
    pepc_BS = model.reactions.PPCc_BS
    pepc_M = model.reactions.PPCc_M
    
    pepc_BS.bounds = (0,0)
    pepc_M.bounds = (0,0)

    #PPDK constraints (Reaction id: PPDKs) (note that this is found in the chloroplast?) 
    #Not detected via immunolocalization but enzyme activity is detected

    ppdks_BS = model.reactions.PPDKs_BS
    ppdks_M = model.reactions.PPDKs_M
    ppdkc_BS = model.reactions.PPDKc_BS
    ppdkc_M = model.reactions.PPDKc_M
    wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                             + ppdks_M.flux_expression
                                             + ppdkc_BS.flux_expression
                                             + ppdkc_M.flux_expression, 
                                             lb = 0, ub = wt_ppdk)
    wt_ppdks_cons.name = 'wt_ppdks_cons'
    model.add_cons_vars(wt_ppdks_cons)
    #Malate Dehydrogenase 
    #Only mitochondrial in WT Rice M cells
    mdhm_M = model.reactions.MDHm_M


    wt_mdh_cons = model.problem.Constraint(mdhm_M.flux_expression,
                                           lb= 0, ub=wt_mdh)
    wt_mdh_cons.name = "wt_mdh_cons"
    model.add_cons_vars(wt_mdh_cons)

    #NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
    #Let's see if I can force it to have a small amount of flux 
    nadp_me_M = model.reactions.MDHys_M
    nadp_me_BS = model.reactions.MDHys_BS

    wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                             + nadp_me_BS.flux_expression,
                                             lb= 0, ub=wt_nadp_me)
    wt_nadpme_cons.name = "wt_nadpme_cons"
    model.add_cons_vars(wt_nadpme_cons)


    #I should add constraints for Carbonic Anhydrase
    #I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
    #Flux is reversible so constraints are bi-directional
    #This should be revised considering that it allows reversible reactions  and an abnormally high flux thru carbonic anhydrase, which shouldn't be the case

    hco3es_m = model.reactions.HCO3Es_M.flux_expression
    hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
    hco3em_m = model.reactions.HCO3Em_M.flux_expression
    hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
    hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
    hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

    ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                       + hco3es_bs + hco3ec_bs + hco3em_bs,
                                      lb = -wt_CA, ub = wt_CA)
    ca_cons.name = 'Carbonic_anhydrase_constraint'
    model.add_cons_vars(ca_cons)


    #Rbcl constaints
    #Retrieve flux expressions oof each RBCl reaction
    rbpc_M = model.reactions.RBPCs_M.flux_expression
    rbpc_BS = model.reactions.RBPCs_BS.flux_expression
    rbpo_M = model.reactions.RBPOs_M.flux_expression
    rbpo_BS = model.reactions.RBPOs_BS.flux_expression

    #Constraint such that it is limited to 132 umol m-2 s-1
    rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)
    rbcl_vcmax_cons.name='rbcl_vcmax_cons'
    model.add_cons_vars(rbcl_vcmax_cons)
    #Constraints for rbcl flux such that v_c/v_o = 3 or higher.
    rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                         - 1*(rbpc_M + rbpc_BS),
                                         lb=0,ub=1000)
    rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
    model.add_cons_vars(rbcl_vcvo)

    #Turn off the RBPC2s reactions since we already defined the constraints above
    model.reactions.RBPC2s_M.bounds = (0,0)
    model.reactions.RBPC2s_BS.bounds = (0,0)
    
    
    
    #What if I simply constrained that of the M cell one to 3:1?
    #This constraint is pretty good actually. 
    #This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
    #wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.
# #This code block is to set a constraint such that M-to-BS cell NGAM ratio is 10-to-1 
# #Similar to what Moreno-Villena et al (2022) had done 

#This function takes two arguments: the model and the maximal  ppfd input to the system
def add_ngam_cons(model, ppfd): 
    ngam_atp_m = mm.get_rxn(model, 'ngam_atp_c_M')
    ngam_atp_bs = mm.get_rxn(model, 'ngam_atp_c_BS')
    ngam_atp_m.bounds = (0,inf)
    ngam_atp_bs.bounds = (0,inf)
    ngam_ratio = mm.set_fix_flux_ratio({ngam_atp_m.id:10, ngam_atp_bs.id:1}, model)
    ngam_ratio.name = 'ngam_BS/M_ratio'
    model.add_cons_vars(ngam_ratio)

    #Retrieve NGAM reactions
    ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c_M')
    ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s_M')
    ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m_M')
    ngam_nadphox_c_BS = mm.get_rxn(model, 'ngam_nadphox_c_BS')
    ngam_nadphox_s_BS = mm.get_rxn(model, 'ngam_nadphox_s_BS')
    ngam_nadphox_m_BS = mm.get_rxn(model, 'ngam_nadphox_m_BS')


    #Set Fixed fluxes
    nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
    nadphox_c_s_M.name = "nadphox_cs_ratio_M"
    nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
    nadphox_s_m_M.name = "nadphox_sm_ratio_M"

    nadphox_c_s_BS = mm.set_fix_flux_ratio({ngam_nadphox_c_BS.id:1, ngam_nadphox_s_BS.id:1},model)
    nadphox_c_s_BS.name = "nadphox_cs_ratio_BS"
    nadphox_s_m_BS = mm.set_fix_flux_ratio({ngam_nadphox_s_BS.id:1, ngam_nadphox_m_BS.id:1}, model)
    nadphox_s_m_BS.name = "nadphox_sm_ratio_BS"

    #Add constraints
    model.add_cons_vars(nadphox_c_s_M)
    model.add_cons_vars(nadphox_s_m_M)
    model.add_cons_vars(nadphox_c_s_BS)
    model.add_cons_vars(nadphox_s_m_BS)

    #Retrieve flux expressionns
    fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
    fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
    fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)

    fex_nadphox_c_BS =  mm.get_flux_exp(model, ngam_nadphox_c_BS)
    fex_nadphox_s_BS =  mm.get_flux_exp(model, ngam_nadphox_s_BS)
    fex_nadphox_m_BS =  mm.get_flux_exp(model, ngam_nadphox_m_BS)

    fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_m)
    fex_atp_c_BS =  mm.get_flux_exp(model, ngam_atp_bs)

    #Set the constraint between ATP:NADPH NGAM to 3:1
    nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M
                                                       + fex_nadphox_c_BS + fex_nadphox_s_BS + fex_nadphox_m_BS) 
                                         - 1*(fex_atp_c_M + fex_atp_c_BS),
                                         lb=0,ub=0)
    nadphox_atpase.name = "nadphox_atpase_ratio"
    model.add_cons_vars(nadphox_atpase)
    #Compute NGAM value and add constraint as a lower bound/upper bound to model
    ngam_value = compute_ngam_atp(ppfd)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    ngam_cons.name = 'NGAM_ATP_constraint'
    model.add_cons_vars(ngam_cons)
    
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above

def print_summary(model, sample_fluxes_df):
    print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
    print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
    print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])
    print('RBPC2s_M', sample_fluxes['RBPC2s_M'], 'RBPC2s_BS', sample_fluxes['RBPC2s_BS'])
    print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC BS', sample_fluxes['PPCc_BS'])
    print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec_M'], 'Carbonic Anhydrase (Cytosolic) BS', sample_fluxes['HCO3Ec_BS'])
    print('NADP-ME M', sample_fluxes['MDHys_M'], 'NADP-ME BS', sample_fluxes['MDHys_BS'])
    print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
    print('Phloem M: ', sample_fluxes['DM_Phloem_M'], 'Phloem BS', sample_fluxes['DM_Phloem_BS'])
    print('co2 consumption M', sample_fluxes['CO2tex_M'], 'co2 consumption BS', sample_fluxes['CO2tex_BS'])
    print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
    print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
    print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
    print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])
    print('ATP synthesis (stromal) M', sample_fluxes['ATPSs_M'], 'ATP synthase (mit) M', sample_fluxes['ATPSm_M'])
    pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
    pd_abs_flux = 0
    for pds in pd_rxn:
        pd_abs_flux += abs(sample_fluxes[pds.id])
    
    print('pd_abs_flux: ', pd_abs_flux)
    
#initialize list of transgenic reactions to add  to model

def add_trans_reactions(model):
    '''
    This function is used to add a number of new tissue-specific reactions that were not present in the
    original model to facilitate modelling of the transgenic C4 rice
    '''
    trans_list = list()
    #Transgenic PEPC copy
    #PEPC = Chloroplastic in M & V (rxn id: PPCc)
    trans_ppcs = Reaction('trans_PPCs_M')
    trans_ppcs.name = "Phosphoenolpyruvate carboxylase, plastidic (Transgenic)"
    
    pep_s0 = model.metabolites.pep_s0
    hco3_s0 = model.metabolites.hco3_s0
    oaa_s0 = model.metabolites.oaa_s0
    pi_s0 = model.metabolites.pi_s0


    #Add metabolites, bounds, and subsystem
    trans_ppcs.add_metabolites({hco3_s0:-1, pep_s0:-1, oaa_s0:1, pi_s0:1})
    trans_ppcs.bounds= model.reactions.PPCc_M.bounds
    trans_ppcs.subsystem = model.reactions.PPCc_M.subsystem

    trans_list.append(trans_ppcs)


    #Transgenic PPDK Copy
    #Since it already exists I'll just copy and readd it
    trans_ppdks_m = Reaction('trans_PPDKs_M')
    trans_ppdks_m.add_metabolites(model.reactions.PPDKs_M.metabolites)
    trans_ppdks_m.bounds = model.reactions.PPDKs_M.bounds
    trans_ppdks_m.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_ppdks_bs = Reaction('trans_PPDKs_BS')
    trans_ppdks_bs.add_metabolites(model.reactions.PPDKs_BS.metabolites)
    trans_ppdks_bs.bounds = model.reactions.PPDKs_BS.bounds
    trans_ppdks_bs.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_list.append(trans_ppdks_m)
    trans_list.append(trans_ppdks_bs)


    #Transgenic NADP-ME
    #NADP-ME = Mitochondrial in M
    trans_nadp_me = Reaction('trans_MDHym_M')

    #retrieve reactants
    mal_m0 = model.metabolites.get_by_id('mal-L_m0')
    nadp_m0 = model.metabolites.nadp_m0
    h_m0 = model.metabolites.h_m0
    nadph_m0 = model.metabolites.nadph_m0
    oaa_m0 = model.metabolites.oaa_m0

    #Add to rxn
    trans_nadp_me.add_metabolites({mal_m0:-1, nadp_m0:-1, h_m0:1, nadph_m0:1, oaa_m0:1})
    #Add bounds
    trans_nadp_me.bounds=(-inf, inf)

    trans_list.append(trans_nadp_me)


    #Malate Dehydrogenase, mitochondrial (M cell)
    trans_MDHm_M = Reaction('trans_MDHm_M')
    trans_MDHm_M.name = 'Malate Dehydrogenase, Mitochondrial'
    trans_MDHm_M.add_metabolites(model.reactions.MDHm_M.metabolites)
    trans_MDHm_M.subsystem = model.reactions.MDHm_M.subsystem

    trans_list.append(trans_MDHm_M)

    #Malate dehydrogenase, plastidic (M cell)
    trans_MDHs_M = Reaction('trans_MDHs_M')
    trans_MDHs_M.name = 'Malate Dehydrogenase, Plastidic'
    trans_MDHs_M.add_metabolites(model.reactions.MDHs_M.metabolites)
    trans_MDHs_M.subsystem = model.reactions.MDHs_M.subsystem

    trans_list.append(trans_MDHs_M)

    #Malate dehydrogenase, plastidic(BS Cell)
    trans_MDHs_BS = Reaction('trans_MDHs_BS')
    trans_MDHs_BS.name = 'Malate Dehydrogenase, Plastidic'
    trans_MDHs_BS.add_metabolites(model.reactions.MDHs_BS.metabolites)
    trans_MDHs_BS.subsystem = model.reactions.MDHs_BS.subsystem

    trans_list.append(trans_MDHs_BS)


    #Trans CA
    #Cytosolic in M
    trans_hco3ec_M = Reaction('trans_hco3ec_M')
    trans_hco3ec_M.name = 'carbonic anhydrase, cytosolic'
    trans_hco3ec_M.add_metabolites(model.reactions.HCO3Ec_M.metabolites)
    trans_hco3ec_M.bounds = model.reactions.HCO3Ec_M.bounds

    trans_hco3ec_M.subsystem = model.reactions.HCO3Ec_M.subsystem
    trans_list.append(trans_hco3ec_M)


    #Bulk add to model
    model.add_reactions(trans_list)
    
    model.repair()
####ADDING TRANS CONSTRAINTS

def add_trans_constraints(model,
                         trans_pepc_rates = 7.01,
                         trans_ppdks_rates = 3.66,
                         trans_mdh_rates = 152.87,
                         trans_nadp_me_rates = 0.60,
                         trans_CA_rates = 8):
    '''
    This function is used to add another layer of constraints to parametize model based on the
    Enzyme reaction rates assayed from Ermakova et al (2021) where the locations are based on the 
    each of the transgenic enzyme's tissue-specific localizations. 
    '''
    
    #PEPC constraint
    wt_PPCc_M = mm.get_rxn(model, 'PPCc_M')
    wt_PPCc_BS = mm.get_rxn(model, 'PPCc_BS')
    trans_PPCs_M = mm.get_rxn(model, 'trans_PPCs_M')                           
    trans_PEPC_cons = model.problem.Constraint(trans_PPCs_M.flux_expression
                                            +wt_PPCc_BS.flux_expression 
                                            + wt_PPCc_M.flux_expression, 
                                            lb = 0, ub = trans_pepc_rates)

    model.add_cons_vars(trans_PEPC_cons)

    #PPDK constraint
    trans_PPDKs_M  = mm.get_rxn(model, 'trans_PPDKs_M')
    trans_PPDKs_BS = mm.get_rxn(model, 'trans_PPDKs_BS')
    wt_PPDKs_M = mm.get_rxn(model, 'PPDKs_M')
    wt_PPDKs_BS = mm.get_rxn(model, 'PPDKs_BS')
    
    trans_PPDKs_cons = model.problem.Constraint( 
        trans_PPDKs_BS.flux_expression + trans_PPDKs_M.flux_expression 
        +wt_PPDKs_BS.flux_expression + wt_PPDKs_M.flux_expression, 
                                             lb = 0, ub = trans_ppdks_rates)
    trans_PPDKs_cons.name = 'trans_ppdks_cons'
    model.add_cons_vars(trans_PPDKs_cons)


    #Malate Dehydrogenase Constraints
    trans_MDHm_M = mm.get_rxn(model, 'trans_MDHm_M')
    trans_MDHs_M = mm.get_rxn(model, 'trans_MDHs_M')
    trans_MDHs_BS = mm.get_rxn(model, 'trans_MDHs_BS')
    wt_MDHm_M =  mm.get_rxn(model, 'MDHm_M')
    wt_MDHs_M = mm.get_rxn(model, 'MDHs_M')
    wt_MDHs_BS = mm.get_rxn(model, 'MDHs_BS')
    
    trans_mdh_cons =  model.problem.Constraint(
       trans_MDHm_M.flux_expression + 
        wt_MDHm_M.flux_expression + 
        trans_MDHs_M.flux_expression + 
        trans_MDHs_BS.flux_expression +
        wt_MDHs_BS.flux_expression +
        wt_MDHs_M.flux_expression, 
        lb= 0, ub=trans_mdh_rates)

    trans_mdh_cons.name = "trans_mdh_cons"
    model.add_cons_vars(trans_mdh_cons)

    #Add NADP-ME constraints
    trans_MDHym_M = mm.get_rxn(model, 'trans_MDHym_M')
    wt_MDHys_M = mm.get_rxn(model, 'MDHys_M')
    wt_MDHys_BS = mm.get_rxn(model, 'MDHys_BS')
    
    trans_nadpme_cons = model.problem.Constraint(
        trans_MDHym_M.flux_expression + 
        wt_MDHys_M.flux_expression + 
        wt_MDHys_BS.flux_expression,
        lb= 0, ub=trans_nadp_me_rates)
    
    trans_nadpme_cons.name = "trans_nadpme"
    model.add_cons_vars(trans_nadpme_cons)

    #Add carbonic anhydrase constraints

    trans_hco3ec_M = mm.get_rxn(model, 'trans_hco3ec_M')
    wt_hco3ec_M = mm.get_rxn(model, 'HCO3Ec_M')
    wt_hco3em_M = mm.get_rxn(model, 'HCO3Em_M')
    wt_hco3es_M = mm.get_rxn(model, 'HCO3Es_M')
    wt_hco3ec_BS = mm.get_rxn(model, 'HCO3Ec_BS')
    wt_hco3em_BS = mm.get_rxn(model, 'HCO3Em_BS')
    wt_hco3es_BS = mm.get_rxn(model, 'HCO3Es_BS')
    
    trans_ca_cons = model.problem.Constraint(trans_hco3ec_M.flux_expression + 
                                             wt_hco3es_M.flux_expression + 
                                             wt_hco3ec_M.flux_expression + 
                                             wt_hco3em_M.flux_expression + 
                                             wt_hco3es_BS.flux_expression + 
                                             wt_hco3ec_BS.flux_expression + 
                                             wt_hco3em_BS.flux_expression,
                                      lb = -trans_CA_rates, ub = trans_CA_rates)
    trans_ca_cons.name = 'Trans_CA_cons'
    model.add_cons_vars(trans_ca_cons)
    model.repair()
    



In [8]:

def generate_flux_samples(model, num_samples, batch_size, output_filename ,thinning,output_dir='./flux_results/flux_sampling/'):
    #This function is used to initialize a flux sampler and afterwards generate a csv file containing the sample solutions.
    #default batch size is 1000

    #Generate sampler
    print("generating sampler for model")
    sampler = sampling.OptGPSampler(model, processes=7, thinning=thinning)
    
    
    #Define output file
    output_dir = str(output_dir)
    output_filename = str(output_filename)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    print('saving output to ', f"{output_dir}/{output_filename}")



    for i in range(num_samples // batch_size):
        print(f"Generating batch {i+1}/{num_samples//batch_size}")
        samples = sampler.sample(n=batch_size)
        df = pd.DataFrame(samples, columns=model.reactions.list_attr("id"))
        if i == 0:
            df.to_csv(f"{output_dir}/{output_filename}", index=False)
        else:
            df.to_csv(f"{output_dir}/{output_filename}", index=False, header=False, mode="a")
#             
            
            
        
    

def parametrize_model(model, ppfd_low, ppfd_high, co2,if_trans, filter_list=list()):
    
    print('start time for parametrize_model(): ' ,datetime.datetime.now())

    #Generates a parametrized model which returns the parametrized model as a callable object.
    
    #Copy model 
    model_instance = model.copy()
                                                                                                                                                                                                                                
    model_instance.medium = define_model_medium(model_instance, co2=co2, o2=inf, ppfd=inf, h=inf, nh4=inf, no3=inf)
    turn_off_cofac_cycles(model_instance)
    add_tissue_constraints(model_instance)
    add_enzyme_constraints(model_instance)

    #Adds NGAM (Computed as an average between the high and low instead of directly constraining it to the model, w/c makes the problem non-linear)
    add_ngam_cons(model_instance, (ppfd_high+ppfd_low)/2)

    #Constrain PPFD range to indicated value
    model_instance.reactions.get_by_id('EX_photonVis(e)').bounds = (-1*ppfd_high, -1*ppfd_low)
    

    #Check if trans then add constraints if true
    if if_trans==True:
        add_trans_reactions(model_instance)
        add_trans_constraints(model_instance)
    
    
    #Readd objective coefficient, maybe it'll work?
    model_instance.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1
    
    #Run FVA to preprocess the model to fix reaction reversibilities as well as to ensure that there are no "extreme" fluxes in the final sampling
    # Perform Flux Variability Analysis (FVA)
    #This step constrains the upper and lower bounds to the detected "Maximal" and "minimal" fluxes given an objective
    #The default fraction of optimum will be implemented
    #Will implement pFBA factor to constrain the model to 110% of the detected lowest flux 
    
    #flux_variability_analysis produces a Pandas Dataframe that can be taken apart and applied to the model_instance as direct bounds. 
    fva_result = cobra.flux_analysis.flux_variability_analysis(model_instance, pfba_factor=1.1, processes=7)

    # Set FVA constraints in the model
    for reaction_id, bounds in fva_result.iterrows():
        reaction = model_instance.reactions.get_by_id(reaction_id)
        # Add tolerance to handle numerical precision issues
        tol = model_instance.tolerance
        reaction.bounds = (bounds['minimum'] - tol, bounds['maximum'] + tol)
        #No issues so far with adding a tolerance value)
        
    #    # Identify and prune blocked reactions
    #I don't think this is necessary anymore since I'm filtering the reactions anyway
    blocked_reactions = cobra.flux_analysis.variability.find_blocked_reactions(model_instance, processes=7, zero_cutoff=tol)
    model_instance.remove_reactions(blocked_reactions)
    
    
    #Remove reactions that are not in the filtered list

    print(len(model_instance.reactions))
    
    if len(filter_list) > 0:
        filt_rxns = list()
        for rxns in model_instance.reactions:
            if rxns.id not in filter_list:
                filt_rxns.append(rxns.id)

        model_instance.remove_reactions(filt_rxns)
    
    print(len(model_instance.reactions))
    
    # Identify unneeded metabolites
    unneeded_metabolites = []
    for metabolite in model.metabolites:
        if metabolite.reactions == []:
            unneeded_metabolites.append(metabolite)

    # Remove unneeded metabolites from the model
    model.remove_metabolites(unneeded_metabolites)
    
    
    
    print('end time for parametrize_model(): ' ,datetime.datetime.now())

    
    return model_instance



def save_model_bounds(model, filename, directory='./flux_results/flux_sampling/model_bounds/'):
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Get the bounds of each reaction in the model
    bounds = []
    for reaction in model.reactions:
        bounds.append([reaction.id, reaction.bounds[0], reaction.bounds[1]])

    # Create a DataFrame with the bounds
    bounds_df = pd.DataFrame(bounds, columns=['Reaction', 'Lower Bound', 'Upper Bound'])

    # Write the DataFrame to a CSV file
    filepath = os.path.join(directory, filename)
    bounds_df.to_csv(filepath, index=False)

    print(f"Model bounds saved to: {filepath}")



In [ ]:
%timeit


#Parametrize models per type by generating model instance
print('Generating parametrized models for flux sampling')
WT_250 =  parametrize_model(wt_model, 225,275,co2=29, if_trans=False)
WT_750 = parametrize_model(wt_model, 725, 775,co2=29, if_trans=False)
WT_1500 = parametrize_model(wt_model, 1475, 1525,co2=29, if_trans=False)

TR_250 = parametrize_model(wt_model, 225, 275,co2=22.2, if_trans=True)
TR_750 = parametrize_model(wt_model, 725, 775,co2=22.2, if_trans=True)
TR_1500 = parametrize_model(wt_model, 1475, 1525,co2=22.2, if_trans=True)

#Generate list of models for flux sampling
sampling_list = [WT_250, WT_750, WT_1500, TR_250, TR_750, TR_1500]

print('generating flux samples for parametrized models')
for model in sampling_list:
    print('start time for sampler generation: ' ,datetime.datetime.now())
    model_name = str([k for k, v in locals().items() if v == model][0])
    name_for_file = str('flux_sample_'+model_name)
    
    #Output the flux bounds of the model to ./fva_bounds/name_for_file
    save_model_bounds(model, filename=name_for_file)
    
    generate_flux_samples(model, num_samples =5000, batch_size=2500, thinning=35000, output_filename=name_for_file)
    del model #Deletes model instance to free up memory 
    print('end time: ', datetime.datetime.now())
    

Generating parametrized models for flux sampling
start time for parametrize_model():  2023-06-15 20:28:44.509744
Read LP format model from file /tmp/tmpmk7lok_l.lp
Reading time = 0.02 seconds
: 3956 rows, 9880 columns, 42914 nonzeros


/home/articulatus/anaconda3/envs/RiceMM/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'numeric'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/articulatus/anaconda3/envs/RiceMM/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


3786
3786
end time for parametrize_model():  2023-06-15 20:30:18.812996
start time for parametrize_model():  2023-06-15 20:30:18.813263
Read LP format model from file /tmp/tmpkrymbyv7.lp
Reading time = 0.02 seconds
: 3956 rows, 9880 columns, 42914 nonzeros


/home/articulatus/anaconda3/envs/RiceMM/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'numeric'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/articulatus/anaconda3/envs/RiceMM/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


3760
3760
end time for parametrize_model():  2023-06-15 20:31:57.088576
start time for parametrize_model():  2023-06-15 20:31:57.089130
Read LP format model from file /tmp/tmprg3yzt5_.lp
Reading time = 0.02 seconds
: 3956 rows, 9880 columns, 42914 nonzeros


...

3754
3754
end time for parametrize_model():  2023-06-15 20:35:21.463132
start time for parametrize_model():  2023-06-15 20:35:21.463448
Read LP format model from file /tmp/tmpfonw6x2h.lp
Reading time = 0.02 seconds
: 3956 rows, 9880 columns, 42914 nonzeros


/home/articulatus/anaconda3/envs/RiceMM/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


3770
3770
end time for parametrize_model():  2023-06-15 20:36:59.025677
start time for parametrize_model():  2023-06-15 20:36:59.025954
Read LP format model from file /tmp/tmpbpgocqow.lp
Reading time = 0.02 seconds
: 3956 rows, 9880 columns, 42914 nonzeros


/home/articulatus/anaconda3/envs/RiceMM/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


3890
3890
end time for parametrize_model():  2023-06-15 20:38:53.445091
generating flux samples for parametrized models
start time for sampler generation:  2023-06-15 20:38:53.445599
Model bounds saved to: ./flux_results/flux_sampling/model_bounds/flux_sample_WT_250
generating sampler for model
Read LP format model from file /tmp/tmpqzsc3dub.lp
Reading time = 0.01 seconds
: 3970 rows, 7578 columns, 32242 nonzeros


In [9]:
%timeit
#This code block is for a version of the above but it filters the model first then afterwards resamples the model

#Load filter list 
filter_list = []

def run_sampler_script1():
    # Load the CSV file and store its contents into the 2-dimensional list
    with open('./misc/filtered_reactions_from_sampling.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            filter_list.append(row)


    #Parametrize models per type by generating model instance
    print('Generating parametrized models for flux sampling')
    WT_250 =  parametrize_model(wt_model, 225,275,co2=29, filter_list=filter_list[0],if_trans=False)
    WT_750 = parametrize_model(wt_model, 725, 775,co2=29, filter_list=filter_list[1],if_trans=False)
    WT_1500 = parametrize_model(wt_model, 1475, 1525,co2=29, filter_list=filter_list[2], if_trans=False)

    TR_250 = parametrize_model(wt_model, 225, 275,co2=22.2, filter_list=filter_list[3],if_trans=True)
    TR_750 = parametrize_model(wt_model, 725, 775,co2=22.2, filter_list=filter_list[4],if_trans=True)
    TR_1500 = parametrize_model(wt_model, 1475, 1525,co2=22.2, filter_list=filter_list[5],if_trans=True)

    #Generate list of models for flux sampling
    sampling_list = [WT_250, WT_750, WT_1500, TR_250, TR_750, TR_1500]
    
    
    #Generate sampler and start the OPTGP sampler
    print('generating flux samples for parametrized models')
    for model in sampling_list:
        print('start time for sampler generation: ' ,datetime.datetime.now())
        model_name = str([k for k, v in locals().items() if v == model][0])
        name_for_file = str('flux_sample_'+model_name+'filtered_75000T')

        #Output the flux bounds of the model to ./fva_bounds/name_for_file
        save_model_bounds(model, filename=name_for_file)

        generate_flux_samples(model, num_samples =5000, batch_size=5000, thinning=100000, output_filename=name_for_file)
        del model #Deletes model instance to free up memory 
        print('end time: ', datetime.datetime.now())


In [ ]:
run_sampler_script1()

Generating parametrized models for flux sampling
start time for parametrize_model():  2023-06-15 08:40:06.705128
Read LP format model from file /tmp/tmpgp_x9qyq.lp
Reading time = 0.02 seconds
: 3956 rows, 9880 columns, 42914 nonzeros


Notes: needed to repeat sampling due to the following:
- sampling matrices are too large to load, compensate with the use of thinning instead. Use specifications from Hermann et al (2020) which uses instead a thinning coefficient of 10000 instead of keeping all samples. My original parameters were n = 50000, batch size = 2000 and thinning = 500. 
- Maybe I should use a thinning coefficient of 100000 instead? Since my model has a number of dimensions one exponent higher than the previously benchmarked Arnold Model
    - I've decided to run my model with a thinning coefficient of 25000. Hopefully autocorrelation and convergence wouldn't be much of an issue. Total samples would be equal in turn to 1.25e8 individual sampling points. 

- I needed to reparametrize samples considering that the objective function doesn't apply in flux sampling. INstead of adding an objective coefficient all demand reactions that output biomass are turned off except for "DM_Phloem_BS". What I need to do instead is re-parametrize it to allow photoassimilates to exit at that reaction rather than to others.
- I'll save the format to ".npz" instead of "csv" since I'm running out of memory whenever I'm loading it out of this script. I think the output of the model is sparse enough that I can instead use this format instead.
    - Actually I can save my csv to a normal csv once since I can just adjust the thinning number instead of keeping all samples.

- It is still a question whether my samples are uncorrelated or not. I will run the diagnostic tests after I finish running the rest of the scripts.



Note: May 17 2023
Flux sampling script was interrupted @ TR750 and was rerun at that point.



Notes: 
OPTGP Is shown to be faster than ACHR and also converges faster

Let's try with a 10 samples first with 10 batches

I think the model is too large to be loaded into the flux sampler. 

It initialized after 30 minutes, let's try sampling na. 

It takes 1-2 hours at most to generate 2000 samples with a thinning coefficient of 10000. Extrapolating from that, we need probably 4-6 hours to generate 5000 samples with a TC of 20000. To keep things tractable I'll keep the thinning coefficient at 10000. I can just add more samples if needed.


Things to add to the paper:


Flux sampling is another constraints-based technique used for characterizing the null space of a given stoichiometric model, providing us insights on the flux distributions of a given metabolic model without the explicit definition of a given cellular objective. Not only does this technique offer advantages over conventional FBA and FVA where it allows researchers to determine the feasibility of solutions within a given defined range, as well as the distribution of these fluxes provided the model constaints.. The latter, in turn, allows statistical analysis to determine whether any given fluxes exist within a single probability distribution or not. This, in turn, allows a more direct comparison of fluxes and to adequately sample a solution space provided the constraints implemented in the model.

The same constraints have been implemented as with the previous set-ups but with a modification with regards to how the objective functions are concerned. Unlike in pFBA where we can set a particular reaction as an objective, no such setting exists for flux sampling. Thus, this setup is in turn reliant on explicitly defined constraints that would define the n reaction-dimensional solution space. To parametrize this, we have turned off all biomass-related demands such as Coleoptile Biomass, Straw_Biomass and the "DM_Phloem_M" reaction to force the model to output flux to the "DM_Phloem_BS" reaction. Additionally, the model is constrained in a similar manner as with the previous pFBA setups that we had done to benchmark and assess our model's performance.

THe benefits of using Flux Sampling compared to other methods such as FVA in characterizing a solution space is that:
a. We may establish confidence intervals with the use of statistical tests to fully characterize a given solution space and whether
b. Rather than simply generating representative fluxes showing the maximum and minimum amount of possible flux towards a particular reaction, flux sampling allows summary statistics to be generated, including the probability distribution of each particular reaction. 
c. Lastly, flux sampling allows a detailed comparison of the interdependence of each particular reaction, showing which reactions are coupled to each other in a positive and negative fashion. This is done

Three setups per parametrized model were initialized, each with a representative light treatment of 250, 750 and 1500 PPFD with a defined range of +-25, respectively. The NGAM reactions were instead set at a static value at each representative light point detailed above. A total of 6 treatments were initialized to represent both the WT and Trans models at the selected light points. 

The sampling algorithm used was the OPTGP (optimized general parallel) sampler from the Cobrapy.sampling package. The sampler was parametrized to generate 5000 samples with a thinning coefficient of 25000, representing 1.25x10^8 sampling points, and was multithreaded into 7 processes each. Run times varied from between 3-6 hours per model parametrized, which was expected based on the number of reactions in the parametrized model.


After the generation of individual sampling points, convergence statistics were done to assess the level of autocorrelation as well as to assess whether the samplers have sufficiently converged to a singular value. The former is done with the use of the "acf" function to assess the level of autocorrelation on each given column, while the latter is assessed with the use of two specific methods -- the Gelman-Rubin and the Geweke statistics. These are two methods that we will use to assess whether the number of sampling points is sufficient to ensure adequate convergence of each reaction in the model. Reactions that have not converged sufficiently will be indicated.

After the assessment of autocorrelation and convergence, each of the pairs between light treatments (225-725, 725-1475, 225-1475) and between models (WT and Trans) are subjected to pairwise Kruskal-Wallis tests to assess which reaction pairs are derived from the same distribution. It is a non-parametric rank-based test test suitable for comparing outputs where there are no assumed distributions behind a population, and has been used in a similar fashion in previous flux sampling setups that involve Plant Metabolism  (Herrmann et al., 2019).  

This procedure, in turn, allows us to deduce in finer detail which reactions in particular are positively correlated with CO2 flux into the BS cell's Rubisco reaction. In the previous pFBA experiment we were able to demonstrate a similar approach to this by performing a sensitivity analysis on Glycine Decarboxylase, and had demonstrated the negative relationship between M cell GLYDHD and BS Cell GLYDHD.

Lastly, we can afterwards assess which fluxes are coupled by iterating over the list of fluxes in a single flux sample matrix and computing the spearman correlations between the pairwise comparisons. From here we can assess which particular reactions are positively or negatively correlated or not. A particular focus will then be held. Further Downstream analysis includes determination of reactions that are positively or negatively coupled with each other and corroborating this fact on whether the reaction is disrupted in the "Trans" parametrized model.  



May 19, 2023
Some observations on the fluxes obtained from initial flux sampling runs:
- It seems it doesn't maximize CO2 assimilation as with pFBA. 
- It features some reactions with infeasibly large fluxes, particularly those expected to have flux cycling.

I asked the Gitter group on what are their thoughts on how to approach this.

One solution I think is this:
- Reparametrize a model first and generate FVA solutions to "pre-process" the model, and in turn add directionality and bound constraints to the model to reduce its solution space further.
- Try readding the objective function for photoassimilate generation as well as the pfba objective, as well as add the "cyclefreeflux" function by Desouki et al (2015).



I am currently testing the Latter.

May 20, 2023

The latter does not work since it applies MILP, I think. It returns the following error:
    
    TypeError: Sampling does not work with integer problems.

I will instead to the former instead. I've also implemented pruning to remove any unused reactions and metabolites based on the find_blocked_reactions() function of FVA. This will further constrain the model to sort of "Contextualise" it. 


May 20, 2023

Based on the Geweke Statistic, and with a z-score of 1.96 (indicating 0.95 confidence  interval) most of the samples have converged on a single statistic. Based on the Gelman Rubin statistic however all of the reactions have not converged to a singular solution. Why?

I've rerun the diagnostic scripts in R using Coda and have produced a more reliable measure of convergence and autocorrelation. In both cases only 

I'll test 5000x25000 samples with the reduced and pFVA-constrained models. I think sampling will be faster considering that I've pruned the samples as well as reduced the solution space by a lot. 


Flux sampling convergence statistics are based on the methods highlighted by Hermmann et al (2019) and by Fallahi et al (2020). It says that we shouldn't use the normal Gelman-Rubin statistic and instead use the Brooks-Gelman formulation instead.

However, for both cases I've instead used the Raftery-Lewis statistic and 
the Geweke Diagnostic to assess convergences for all parameters. I decided to re-run instead the two last samples considering that they both have significant amounts of reactions that haven't converged based on the RL statistic and the GW statistic.

For the RL statistic both 250 and 750 parametrized samplers have converged, although the Geweke diagnostic only reports a convergence rate of around 70 percent. In 1500 the convergence rate falls to 40 percent only which necessitates the re-run scenario. AFterwards I can simply re-run the scripts and re-assess my results. In the meantime however I can analyze both 250 and 750 scenarios as well the highb light scenarios particularly those with significantly varying distributions


June 13, 2023

I have rerun both WT andTR 1500 to validate if both have converged. If it hasn't I'll report the results as-is and include it in the discussion.

June 15, 2023

I have an idea in order to ensure convergence as well as to reduce runtimes.
My idea is to remove all reactions that are not foundd in the model to 0 in order to reduce the nullspace of the model. This will be based on the initial runs for the flux sampling runs. 

Afterwards I need to compare their distributions based dun sa mga previous runs to determine if pareho yung distribution. If it is the same or virtually the samme I will use it because I can ensure that it has a higher convergence rate than the one with lower samples.

If in case this works then it should return the same distribution and my samples would've converged faster. Furthermore there isn't need to go for breadth considering that my parametrizations are already fixed. Hopefully it can converge faster but since I've changed the thinning constant to 100k it may change the distribution.









Pipeline breakdown:

Load CSVs to a memory saving format first

1.
Run convergence statistics on each and generate plots to assess total convergence stats for each CSV. These will include tests such as the 
Raftery-Lewis statistic and the Geweke statistics to assess both autocorrelation as well as assess convergence.

Afterwards get only the flux names of those reactions that have converged
Run pairwise Kruskal-wallis tests per CSV using the above list of converged reactions
Identify each reaction with significant and non-significant distributions each

Generate histograms/probability densities for relevant reactions with significantly different distributions with WT and Trans models.
2. Flux coupling analysis
Check which fluxes are coupled with each otehr and identify which fluxes are then related to each other, particularly Carbon Fixation reactions in the BS cell such as Rubisco and the DM_Phloem reactions
